# NESDB Combined Database Notebook
This notebook fetches the NES pattern HTML, parses it to extract NESdb annotations, merges with the original CRM1 CSV to include protein names, full sequences, and headers, and saves a combined database.

## 1. Download the NES pattern HTML

In [ ]:
import requests

# Download the NES pattern page
url = "http://prodata.swmed.edu/nes_pattern_location/"
resp = requests.get(url)
resp.raise_for_status()

# Save HTML locally
with open("data/nes_pattern_location.html", "w", encoding="utf-8") as f:
    f.write(resp.text)

print("Downloaded NES pattern HTML to nes_pattern_location.html")

## 2. Parse HTML and build annotation DataFrame

In [2]:
import pandas as pd
import re
from bs4 import BeautifulSoup

# Read and parse the saved HTML
with open("data/nes_pattern_location.html", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# Load the main table
html_df = pd.read_html(str(soup))[0]
html_df.columns = html_df.columns.str.strip()

# Extract numeric NESdb_ID (nullable Int64)
ids = html_df['refDB'].str.extract(r'NESdb:(\d+)')[0]
html_df['NESdb_ID'] = ids.astype('Int64')

# Extract UniProt ID column from HTML (adjust column name as needed)
if 'uniprotID' in html_df.columns:
    html_df['uniprotID'] = html_df['uniprotID']
elif 'UniProt ID' in html_df.columns:
    html_df['uniprotID'] = html_df['UniProt ID']
else:
    html_df['uniprotID'] = pd.NA  # placeholder if missing

# Extract true peptide sequence (letters only)
html_df['true_sequence'] = html_df['sequence'].str.extract(r'^([A-Z]+)')[0]

# Extract start and compute end position
html_df['start'] = html_df['start#'].astype(int)
html_df['end'] = html_df['start'] + html_df['true_sequence'].str.len() - 1

# Drop rows without a valid NESdb_ID
html_df = html_df.dropna(subset=['NESdb_ID'])

# Build the annotation DataFrame
annotation_df = (
    html_df[['NESdb_ID', 'uniprotID', 'true_sequence', 'start', 'end']]
    .drop_duplicates()
)

print(f"Prepared annotation for {len(annotation_df)} unique NESdb entries.")

C:\Users\shay3\AppData\Local\Temp\ipykernel_19064\2899887163.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  html_df = pd.read_html(str(soup))[0]


Prepared annotation for 381 unique NESdb entries.


## 3. Load original CRM1 CSV and extract metadata

In [3]:
import pandas as pd

# Load the original CSV with full protein sequences and headers
df = pd.read_csv("data/NesDB_all_CRM1_with_peptides_train.csv")

# Extract NESdb_ID from the 'ID' column
df['NESdb_ID'] = (
    df['ID']
      .str.extract(r'NES ID:\s*(\d+)')[0]
      .astype(int)
)

# Keep only the metadata columns and rename 'Full Name' to 'name'
df_meta = df[['NESdb_ID', 'ID', 'Sequence', 'Fasta Header']].rename(columns={'ID': 'name'})

print(f"Loaded original CSV metadata for {len(df_meta)} entries.")

Loaded original CSV metadata for 351 entries.


## 4. Merge annotation with metadata and save

In [4]:
# Merge annotation_df with df_meta on NESdb_ID
combined_db = (
    annotation_df
    .merge(df_meta, on='NESdb_ID', how='left')
)

# Reorder columns to:
# NESdb_ID, uniprotID, name, true_sequence, start, end, Sequence, Fasta Header
combined_db = combined_db[[
    'NESdb_ID',
    'uniprotID',
    'name',
    'true_sequence',
    'start',
    'end',
    'Sequence',
    'Fasta Header'
]]

# remove duplicated rows and rows with missing values
combined_db = combined_db.drop_duplicates(keep='first')
combined_db = combined_db.dropna()

# Save the combined database
combined_db.to_csv("NESDB_combined_database.csv", index=False)

print(f"Saved combined database with {len(combined_db)} rows to NESDB_combined_database.csv")

Saved combined database with 560 rows to NESDB_combined_database.csv
